In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from scipy.stats import pearsonr
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import timedelta
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Activation

from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

handler = logging.FileHandler('multi_nn.log')
handler.setLevel(logging.INFO)
logger.addHandler(handler)

%matplotlib inline
init_notebook_mode(connected = True)

Using TensorFlow backend.
/home/rbkn99/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning:

inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()



In [2]:
class PPPredictor:
    
    def __init__(self):
        self.model = None
        self.week_shift = 13
    
    def prepare_data(self, data, actual_date):
        df = data.copy()
        df.index = pd.to_datetime(df['Date'])
        df.drop(['Date'], axis=1, inplace=True)
        for col in df.columns:
            df[col].interpolate(method='time', inplace=True)
            df[col].fillna(method='bfill', inplace=True)
            
        df.index = pd.to_datetime(df.index)
        return df[df.index <= pd.to_datetime(actual_date)]

    def fit(self, prepared_data, use_text_model=False):
        model = Sequential()

        model.add(LSTM(
            input_dim=7,
            output_dim=50,
            return_sequences=True))
        model.add(Dropout(0.2))

        model.add(LSTM(
            100,
            return_sequences=False))
        model.add(Dropout(0.2))

        model.add(Dense(
            output_dim=1))
        model.add(Activation('linear'))

        model.compile(loss='mse', optimizer='rmsprop')
        
        df = prepared_data.drop(['PPSpotAvgPrice'], axis=1)
        df['Date'] = pd.to_numeric(df.index)
        
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(df)
        self.X_train = X_train
        
        X_train = X_train[:-self.week_shift]

        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

        self.scaler_y = MinMaxScaler()
        y_train = np.reshape(np.array(prepared_data['PPSpotAvgPrice']), (-1, 1))
        y_train = self.scaler_y.fit_transform(y_train)
        y_train = y_train[self.week_shift:, 0]
        
        model.fit(X_train, y_train, batch_size=128, nb_epoch=10)
        
        self.model = model
        return model

    
    def predict(self, date):
        X_test = self.X_train[-1]
        X_test = np.reshape(X_test, (1, 1, X_test.shape[0]))
        forecast = self.model.predict(X_test)
        return self.scaler_y.inverse_transform(forecast)

In [3]:
 def get_next_monday(df, date):
    first = True
    while len(df[df.index == str(date).split()[0]]) == 0:
        if first:
            date += timedelta(days=((7 - date.weekday()) % 7))
            first = False
        else:
            date += timedelta(7)
    return date

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def test_sol(date):
    df = pd.read_csv('./data/retrieved_data.csv')
    ppp = PPPredictor()
    prepared = ppp.prepare_data(df, date)
    ppp.fit(prepared)
    """
    df.index = pd.to_datetime(df['Date'])
    df.drop(['Date'], axis=1, inplace=True)
    for col in df.columns:
        df[col].interpolate(method='time', inplace=True)
        df[col].fillna(method='bfill', inplace=True)"""
    
    start = pd.to_datetime(date) + timedelta(3 * 30)
    end = pd.to_datetime(date) + timedelta(4 * 30)
    pred = []
    actual = []
    for dt in pd.date_range(start, end, freq='W'):
        pred.append(ppp.predict(str(dt).split()[0]))
        actual_dt = get_next_monday(df, dt)
        right_val = df[df.index == str(actual_dt).split()[0]]['PPSpotAvgPrice'].iloc[0]
        actual.append(right_val)
    logger.info('MAPE for {}: {:.2f}%'.format(date, mean_absolute_percentage_error(pred, actual)))

for dt in pd.date_range(start=pd.to_datetime('2015-12-01'), 
                        end=pd.to_datetime('2018-07-01'), freq='M'):
    test_sol(str(dt).split()[0])

Epoch 1/10
98/98 [==============================] - 2s 23ms/step - loss: 0.3409
Epoch 2/10
98/98 [==============================] - 0s 73us/step - loss: 0.3139
Epoch 3/10
98/98 [==============================] - 0s 72us/step - loss: 0.2934
Epoch 4/10
98/98 [==============================] - 0s 44us/step - loss: 0.2738
Epoch 5/10
98/98 [==============================] - 0s 90us/step - loss: 0.2575
Epoch 6/10
98/98 [==============================] - 0s 90us/step - loss: 0.2419
Epoch 7/10
98/98 [==============================] - 0s 183us/step - loss: 0.2253
Epoch 8/10
98/98 [==============================] - 0s 107us/step - loss: 0.2052
Epoch 9/10
98/98 [==============================] - 0s 224us/step - loss: 0.1953
Epoch 10/10
98/98 [==============================] - 0s 156us/step - loss: 0.1778


INFO:__main__:MAPE for 2015-12-31: 3.90%


Epoch 1/10
102/102 [==============================] - 2s 18ms/step - loss: 0.3301
Epoch 2/10
102/102 [==============================] - 0s 76us/step - loss: 0.3017
Epoch 3/10
102/102 [==============================] - 0s 97us/step - loss: 0.2802
Epoch 4/10
102/102 [==============================] - 0s 77us/step - loss: 0.2635
Epoch 5/10
102/102 [==============================] - 0s 123us/step - loss: 0.2466
Epoch 6/10
102/102 [==============================] - 0s 429us/step - loss: 0.2280
Epoch 7/10
102/102 [==============================] - 0s 168us/step - loss: 0.2117
Epoch 8/10
102/102 [==============================] - 0s 139us/step - loss: 0.1966
Epoch 9/10
102/102 [==============================] - 0s 112us/step - loss: 0.1815
Epoch 10/10
102/102 [==============================] - 0s 165us/step - loss: 0.1638


INFO:__main__:MAPE for 2016-01-31: 2.53%


Epoch 1/10
106/106 [==============================] - 3s 24ms/step - loss: 0.3113
Epoch 2/10
106/106 [==============================] - 0s 38us/step - loss: 0.2867
Epoch 3/10
106/106 [==============================] - 0s 66us/step - loss: 0.2675
Epoch 4/10
106/106 [==============================] - 0s 75us/step - loss: 0.2483
Epoch 5/10
106/106 [==============================] - 0s 83us/step - loss: 0.2327
Epoch 6/10
106/106 [==============================] - 0s 89us/step - loss: 0.2193
Epoch 7/10
106/106 [==============================] - 0s 162us/step - loss: 0.2013
Epoch 8/10
106/106 [==============================] - 0s 126us/step - loss: 0.1851
Epoch 9/10
106/106 [==============================] - 0s 191us/step - loss: 0.1692
Epoch 10/10
106/106 [==============================] - 0s 251us/step - loss: 0.1558


...

INFO:__main__:MAPE for 2018-04-30: 16.23%


Epoch 1/10
221/221 [==============================] - 13s 59ms/step - loss: 0.2197
Epoch 2/10
221/221 [==============================] - 0s 67us/step - loss: 0.1764
Epoch 3/10
221/221 [==============================] - 0s 60us/step - loss: 0.1439
Epoch 4/10
221/221 [==============================] - 0s 76us/step - loss: 0.1147
Epoch 5/10
221/221 [==============================] - 0s 57us/step - loss: 0.0898
Epoch 6/10
221/221 [==============================] - 0s 68us/step - loss: 0.0696
Epoch 7/10
221/221 [==============================] - 0s 75us/step - loss: 0.0536
Epoch 8/10
221/221 [==============================] - 0s 93us/step - loss: 0.0427
Epoch 9/10
221/221 [==============================] - 0s 63us/step - loss: 0.0341
Epoch 10/10
221/221 [==============================] - 0s 88us/step - loss: 0.0325


INFO:__main__:MAPE for 2018-05-31: 14.45%


Epoch 1/10
225/225 [==============================] - 14s 62ms/step - loss: 0.2293
Epoch 2/10
225/225 [==============================] - 0s 55us/step - loss: 0.1902
Epoch 3/10
225/225 [==============================] - 0s 66us/step - loss: 0.1601
Epoch 4/10
225/225 [==============================] - 0s 67us/step - loss: 0.1332
Epoch 5/10
225/225 [==============================] - 0s 70us/step - loss: 0.1080
Epoch 6/10
225/225 [==============================] - 0s 72us/step - loss: 0.0868
Epoch 7/10
225/225 [==============================] - 0s 75us/step - loss: 0.0684
Epoch 8/10
225/225 [==============================] - 0s 92us/step - loss: 0.0547
Epoch 9/10
225/225 [==============================] - 0s 76us/step - loss: 0.0435
Epoch 10/10
225/225 [==============================] - 0s 64us/step - loss: 0.0372


INFO:__main__:MAPE for 2018-06-30: 21.60%
